In [1]:
import configparser
import os
from powerbiclient.authentication import DeviceCodeLoginAuthentication
from powerbiclient import Report, models

In [ ]:
device_auth = DeviceCodeLoginAuthentication()
group_id = "e5122c64-1bae-4c59-85ce-35484f92f8df"
report_id = "2f91863f-3a73-4edf-b74b-f011f0debd4e"

In [3]:
report = Report(group_id=group_id, report_id=report_id, auth=device_auth)

In [5]:
print(report)

Report()


In [11]:
# 2. 获取访问令牌（用于 REST API）
access_token = device_auth.get_access_token()

# 3. 调用 Power BI REST API 导出 PDF
export_url = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/reports/{report_id}/ExportTo"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


In [ ]:
print(access_token)

In [ ]:
import requests
#  4. 发起导出请求
response = requests.post(
    export_url,
    headers=headers,
    json={
        "format": "PDF",
        "powerBIReportConfiguration": {
            "settings": {
                "locale": "en-US"  # 可选：设置语言
            },
            "pages": []  # 可选：指定要导出的页面（空数组表示全部）
        }
    }
)

In [ ]:
# 5. 检查导出状态
if response.status_code == 202:
    export_id = response.headers.get("Location").split("/")[-1]
    print(f"导出已开始, ID: {export_id}")
    
    # 6. 轮询导出状态
    status_url = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/reports/{report_id}/exports/{export_id}"
    
    import time
    while True:
        status_response = requests.get(status_url, headers=headers)
        status_data = status_response.json()
        
        if status_data["status"] == "Succeeded":
            print("导出成功！")
            break
        elif status_data["status"] == "Failed":
            print(f"导出失败: {status_data.get('error', '未知错误')}")
            break
        else:
            print(f"导出状态: {status_data['status']}，等待 5 秒...")
            time.sleep(5)
    
    # 7. 下载 PDF
    if status_data["status"] == "Succeeded":
        pdf_url = status_data["resourceLocation"]
        pdf_response = requests.get(pdf_url, headers=headers)
        
        with open("exported_report.pdf", "wb") as f:
            f.write(pdf_response.content)
        print("PDF 已保存到本地！")
else:
    print(f"导出请求失败: {response.status_code} - {response.text}")

导出已开始，ID: Mi9CbG9iSWRWMi04ZjMxNjAyYS1iZDNiLTRhYmMtOTA1OC0yMTIyNGViODE0ZGJWSGliT1BFNU5GWVRJVEh5elA4M3JIWUlHUU5pOTk1MEZBNDBBanVmZ2ZFPS4=
导出成功！
PDF 已保存到本地！


In [17]:
"""
# 筛选器配置示例
filters = {
    "format": "PDF",
    "powerBIReportConfiguration": {
        "settings": {
            "locale": "en-US"
        },
        "identities": [
            {
                "username": "effective_user@domain.com",  # 可选的用户身份
                "roles": ["RoleName"],  # 可选的角色
                "datasets": ["dataset_id"]  # 可选的数据集ID
            }
        ],
        "filters": [
            {
                "filter": "TableName/ColumnName",  # 表名/列名
                "operator": "In",  # 操作符：In, NotIn, Contains, DoesNotContain等
                "values": ["Value1", "Value2"]  # 筛选值
            },
            # 可以添加多个筛选器
            {
                "filter": "AnotherTable/AnotherColumn",
                "operator": "In",
                "values": ["Value3"]
            }
        ]
    }
}
"""

'\n# 筛选器配置示例\nfilters = {\n    "format": "PDF",\n    "powerBIReportConfiguration": {\n        "settings": {\n            "locale": "en-US"\n        },\n        "identities": [\n            {\n                "username": "effective_user@domain.com",  # 可选的用户身份\n                "roles": ["RoleName"],  # 可选的角色\n                "datasets": ["dataset_id"]  # 可选的数据集ID\n            }\n        ],\n        "filters": [\n            {\n                "filter": "TableName/ColumnName",  # 表名/列名\n                "operator": "In",  # 操作符：In, NotIn, Contains, DoesNotContain等\n                "values": ["Value1", "Value2"]  # 筛选值\n            },\n            # 可以添加多个筛选器\n            {\n                "filter": "AnotherTable/AnotherColumn",\n                "operator": "In",\n                "values": ["Value3"]\n            }\n        ]\n    }\n}\n'

In [18]:
"""
基本筛选:

{
    "filter": "Sales[Region]",
    "operator": "In",
    "values": ["East", "West"]
}

多条件组合筛选：
{
    "filter": "(Sales[Region] = 'East' and Sales[Amount] > 1000) or (Sales[Region] = 'West' and Sales[Amount] < 500)"
}

日期范围筛选:
{
    "filter": "Sales[Date]",
    "operator": "In",
    "values": ["2023-01-01", "2023-12-31"]
}

"""


'\n基本筛选:\n\n{\n    "filter": "Sales[Region]",\n    "operator": "In",\n    "values": ["East", "West"]\n}\n\n多条件组合筛选：\n{\n    "filter": "(Sales[Region] = \'East\' and Sales[Amount] > 1000) or (Sales[Region] = \'West\' and Sales[Amount] < 500)"\n}\n\n日期范围筛选:\n{\n    "filter": "Sales[Date]",\n    "operator": "In",\n    "values": ["2023-01-01", "2023-12-31"]\n}\n\n'

### With Filter

In [41]:
# 配置导出参数
export_params = {
        "format": "PDF",
        "powerBIReportConfiguration": {
            "settings": {
                "locale": "en-US"  # 可选：设置语言
            },
            "pages": [],  # 可选：指定要导出的页面（空数组表示全部）
            "filters": [
                {
                    "filter": "financials(2)[Country]",  # 表名/列名
                    "operator": "In",  # 操作符：In, NotIn, Contains, DoesNotContain等
                    "values": ["Canada"]  # 筛选值
                }
            ]
        }
}

#  4. 发起导出请求
response = requests.post(
    export_url,
    headers=headers,
    json=export_params
)

# 5. 检查导出状态
if response.status_code == 202:
    export_id = response.headers.get("Location").split("/")[-1]
    print(f"导出已开始, ID: {export_id}")
    
    # 6. 轮询导出状态
    status_url = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/reports/{report_id}/exports/{export_id}"
    
    import time
    while True:
        status_response = requests.get(status_url, headers=headers)
        status_data = status_response.json()
        
        if status_data["status"] == "Succeeded":
            print("导出成功！")
            break
        elif status_data["status"] == "Failed":
            print(f"导出失败: {status_data.get('error', '未知错误')}")
            break
        else:
            print(f"导出状态: {status_data['status']}，等待 5 秒...")
            time.sleep(5)
    
    # 7. 下载 PDF
    if status_data["status"] == "Succeeded":
        pdf_url = status_data["resourceLocation"]
        pdf_response = requests.get(pdf_url, headers=headers)
        
        with open("exported_report.pdf", "wb") as f:
            f.write(pdf_response.content)
        print("PDF 已保存到本地！")
else:
    print(f"导出请求失败: {response.status_code} - {response.text}")

导出已开始, ID: Mi9CbG9iSWRWMi0wMDdhMzQxNS04Njg2LTQ1ZWItYTUzNi1lNjJjMWJkZDcxYjR5b1Y4WmV5WkhxYWsybkpjazV3eU1OUEtGMnhRLnAwWG5kUC5CLg==
导出状态: Running，等待 5 秒...
导出状态: Running，等待 5 秒...
导出状态: Running，等待 5 秒...
导出成功！
PDF 已保存到本地！
